In [1]:
!pip install minsearch


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import json
from minsearch import AppendableIndex

In [3]:
#from chat_assistant_2 import IPythonChatInterface, Tools, ChatAssistant

In [4]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [44]:
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [45]:
index = AppendableIndex(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [46]:
from openai import OpenAI
client = OpenAI()

In [47]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}
    
    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

search_tool = {
    "type": "function",
    "name": "search",
    "description": "Search the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query text to look up in the course FAQ."
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}


In [48]:
# search_tool = {
#     "type": "function",
#     "function": {
#         "name": "search",
#         "description": "Search the FAQ database",
#         "parameters": {
#             "type": "object",
#             "properties": {
#                 "query": {
#                     "type": "string",
#                     "description": "Search query text to look up in the course FAQ."
#                 }
#             },
#             "required": ["query"],
#             "additionalProperties": False
#         }
#     }
# }


In [72]:
results = search('I just discovered the course.Can I still join now?')
print(results[0]['text'])

Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


In [50]:
tools = [search_tool]

In [51]:
# response = client.responses.create(
#     model="gpt-4o-mini",
#     input="hello"
# )
# print(response.output_text)


In [65]:
developer_prompt = """
You're a course teaching assistant.
You're given a question from a course student and your task is to answer it

Before making any function calls, explain you reasoning behind calling the funtion.

When searching in our FAQ, perform multiple search queries with differently phrased.

In the end also ask a related question to make it more engaging !
""".strip()

#question = "what are the things I need to install and learn to do well in module 1?"


chat_messages = [
    {"role": "developer", "content": developer_prompt},
]

response = client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages, 
    tools=tools
)

In [66]:
response.output[0].type

'message'

In [67]:
response.output

[ResponseOutputMessage(id='msg_0c0e5bc957ec607d006905ec1b173481a29b004fb2735f6566', content=[ResponseOutputText(annotations=[], text='Sure! To help answer the student\'s question, I\'ll start by searching the FAQ database for relevant information. Since I want to ensure I cover various angles and phrasing of the same topic, I will conduct multiple searches with different but related queries.\n\nFor example, if a student asks about assignment deadlines, I might search for:\n1. "What are the deadlines for assignments?"\n2. "When are assignments due?"\n3. "How can I find assignment submission dates?"\n\nBy doing this, I increase the likelihood of finding comprehensive information that addresses the student\'s query effectively.\n\nNow, let\'s search the FAQ database with these varied queries. Could you please provide me with the specific question from the student?', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')]

In [68]:
#response.output[0].name

In [69]:
def make_call(entry):
    args = json.loads(entry.arguments)
    f_name = entry.name
    f = globals()[f_name]  
    call_id = entry.call_id
    results = f(**args)
    output_json = json.dumps(results)
    
    call_output = {
        "type": "function_call_output",
        "call_id": entry.call_id,
        "output": output_json,
    }
    return call_output

In [70]:
while True:
    question = input('User:').strip()

    if question == 'stop':
        print("Chat Ended")
        break

    chat_messages.append({"role": "user", "content": question})
    while True:
        response = client.responses.create(
            model='gpt-4o-mini',
            input=chat_messages, 
            tools=tools
        )

        has_funtion_call = False
        for entry in response.output:
        
            chat_messages.append(entry)
        
            if entry.type == 'message':
                print('Assistant')
                print(entry.content[0].text)
            if entry.type == 'function_call':
                print('Invoking', entry.name, entry.arguments)
                call_output = make_call(entry)
                chat_messages.append(call_output)
                has_funtion_call =  True
        if not has_funtion_call:
            break

User: How do I do well in module 2?


Assistant
To answer your question about doing well in module 2, I want to look up specific strategies or tips related to this module, as well as any common challenges or FAQs that might be addressed in our database.

First, I will perform a search for "how to succeed in module 2" and then follow up with a search for "tips for module 2" or similar phrases. This approach should help me find relevant information tailored to your needs.

Let me get started on that!
Invoking search {"query":"how to succeed in module 2"}
Invoking search {"query":"tips for module 2"}
Assistant
Based on my searches, here are some tips and strategies for doing well in Module 2:

1. **Understand the Concepts**: Make sure you grasp the foundational concepts before diving deeper. Take notes and summarize key points from the lectures.

2. **Practice Regularly**: Engage with the assignments and hands-on labs frequently. The more you practice, the more comfortable you'll become with the material.

3. **Utilize Resour

User: thanks, stop !


Assistant
You're welcome! If you have any more questions or need assistance in the future, feel free to ask. Good luck with your studies!


User: stop


Chat Ended


In [19]:
chat_messages

[{'role': 'developer',
  'content': "You're a course teaching assistant.\nYou're given a question from a course student and your task is to answer it\n\nWhen searching in our FAQ, perform multiple search queries with differently phrased."},
 {'role': 'user',
  'content': 'what are the things I need to install and learn to do well in module 1?'},
 ResponseFunctionToolCall(arguments='{"query":"module 1 requirements"}', call_id='call_TJUaPelCd7yUEhAttSt4gZD4', name='search', type='function_call', id='fc_086e2691b485ad7a006905d2ea11948194830179ad378f6c76', status='completed'),
 {'type': 'function_call_output',
  'call_id': 'call_TJUaPelCd7yUEhAttSt4gZD4',
  'output': '[{"text": "Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.\\nTo create a virtual env and install packages (run only once)\\npython -m venv env\\nsource env/bin/activate\\npip install -r ../requirements.txt\\nTo activate it (you\'ll need to run it every time

In [20]:
responsess = client.responses.create(
    model='gpt-4o',
    input=chat_messages, 
    tools=tools
)

In [28]:
entry = responsess.output[0]

In [29]:
entry

ResponseOutputMessage(id='msg_086e2691b485ad7a006905d4d30fb481948d48c67db24573d2', content=[ResponseOutputText(annotations=[], text="To do well in Module 1 of the course, here are some key things you need to install and learn:\n\n### Install\n1. **Docker**: You'll work with Docker containers, so ensuring Docker is installed and running is essential.\n   \n2. **Terraform**: Installing Terraform is vital as you'll use it for infrastructure management.\n\n3. **PostgreSQL**: You might need to install PostgreSQL, especially for any database interactions.\n\n4. **Python Modules**: Ensure the installation of necessary Python packages such as `psycopg2`. You can install it via pip with `pip install psycopg2-binary`.\n\n5. **Any Required Updates**: For Windows users, ensure that your Windows Terminal, WSL, and Windows Security updates are up-to-date.\n\n### Learn\n- Familiarize yourself with Docker commands and container management.\n- Understand the basics of Terraform for managing infrastruct

In [33]:
entry.content[0].text

"To do well in Module 1 of the course, here are some key things you need to install and learn:\n\n### Install\n1. **Docker**: You'll work with Docker containers, so ensuring Docker is installed and running is essential.\n   \n2. **Terraform**: Installing Terraform is vital as you'll use it for infrastructure management.\n\n3. **PostgreSQL**: You might need to install PostgreSQL, especially for any database interactions.\n\n4. **Python Modules**: Ensure the installation of necessary Python packages such as `psycopg2`. You can install it via pip with `pip install psycopg2-binary`.\n\n5. **Any Required Updates**: For Windows users, ensure that your Windows Terminal, WSL, and Windows Security updates are up-to-date.\n\n### Learn\n- Familiarize yourself with Docker commands and container management.\n- Understand the basics of Terraform for managing infrastructure as code.\n- Learn to connect Python applications to SQL databases like PostgreSQL using libraries like SQLAlchemy.\n\nThese inst

In [7]:
from typing import Any, Dict, List

class CourseFAQTools:

    def __init__(self, index):
        self.index = index

    def search(self, query: str) -> List[Dict[str, Any]]:
        """
        Search the FAQ database for entries matching the given query.
    
        Args:
            query (str): Search query text to look up in the course FAQ.
    
        Returns:
            List[Dict[str, Any]]: A list of search result entries, each containing relevant metadata.
        """
        boost = {'question': 3.0, 'section': 0.5}
    
        results = self.index.search(
            query=query,
            filter_dict={'course': 'data-engineering-zoomcamp'},
            boost_dict=boost,
            num_results=5,
            output_ids=True
        )
    
        return results


    def add_entry(self, question: str, answer: str) -> None:
        """
        Add a new entry to the FAQ database.
    
        Args:
            question (str): The question to be added to the FAQ database.
            answer (str): The corresponding answer to the question.
        """
        doc = {
            'question': question,
            'text': answer,
            'section': 'user added',
            'course': 'data-engineering-zoomcamp'
        }
        self.index.append(doc)


In [8]:
faq_tools = CourseFAQTools(index)

In [9]:
tools = Tools()
tools.add_tools(faq_tools)
tools.get_tools()

[{'type': 'function',
  'name': 'add_entry',
  'description': 'Add a new entry to the FAQ database.\n\nArgs:\n    question (str): The question to be added to the FAQ database.\n    answer (str): The corresponding answer to the question.',
  'parameters': {'type': 'object',
   'properties': {'question': {'type': 'string',
     'description': 'question parameter'},
    'answer': {'type': 'string', 'description': 'answer parameter'}},
   'required': ['question', 'answer'],
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'search',
  'description': 'Search the FAQ database for entries matching the given query.\n\nArgs:\n    query (str): Search query text to look up in the course FAQ.\n\nReturns:\n    List[Dict[str, Any]]: A list of search result entries, each containing relevant metadata.',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string',
     'description': 'query parameter'}},
   'required': ['query'],
   'additionalProperties': False}}]

In [10]:
developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Before making any function calls, explain your reasoning why you want to perform something.

When searching in our FAQ, perform multiple search queries with diffierently phrased questions.

At the end, as the user a question to make it more engaging
""".strip()

interface = IPythonChatInterface()

chat = ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    interface=interface,
    openai_client=client
)

In [11]:
chat.run()

User: how do I do well in module 1?


User: stop


chat ended
